In [2]:
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

2.5.0


In [5]:
class My_model_class():

    def __init__(self):
        '''
        Crear el modelos para segmentación
        '''

    def rgbd_fusion(self,input_r,input_d):
        r_1 = tf.keras.layers.GlobalAveragePooling2D()(input_r)
        r_2 = tf.keras.layers.Conv2D(4, kernel_size=(1,1), activation='ReLU')(r_1)
        r_3 = tf.keras.layers.Conv2D(64, kernel_size=(1,1), activation='Sigmoid')(r_2)
        m_1 = tf.keras.layers.Mul(input_r,r_3)
        
        d_1 = tf.keras.layers.GlobalAveragePooling2D()(input_d)
        r_2 = tf.keras.layers.Conv2D(4, kernel_size=(1,1), activation='ReLU')(r_1)
        r_3 = tf.keras.layers.Conv2D(64, kernel_size=(1,1), activation='Sigmoid')(r_2)
        m_2 = tf.keras.layers.Mul(input_d,d_2)
        
        return tf.keras.layers.Add(m_1,m_2)

    def resnet_layer(self,input_res, filters):
        x = tf.keras.layers.Conv2D(filters, kernel_size=(3,1), activation='relu', padding='same')(input_res)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, kernel_size=(1,3), activation='relu', padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, kernel_size=(3,1), activation='relu', padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, kernel_size=(1,3), activation='relu', padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Add()([x, input_res])
        x = tf.keras.layers.Activation('relu')(x)
        return x

    def context_module(self,input_con):
        c_1 = tf.keras.layers.GlobalAveragePooling2D()(input_con)
        c_2 = tf.keras.layers.AveragePooling2D()(input_con)

        
    def create_model(self):
        input_rgb = tf.keras.layers.Input(shape=(1, 3, 480, 640),dtype=tf.float32)
        input_depth = tf.keras.layers.Input(shape=(1, 1, 480, 640),dtype=tf.float32)
        xr = tf.keras.layers.Conv2D(64,kernel_size=(7,7),strides=(2,2),padding='same',dilation_rate=(1,1),activation='relu')(input_rgb)
        xd = tf.keras.layers.Conv2D(64,kernel_size=(7,7),strides=(2,2),padding='same',dilation_rate=(1,1),activation='relu')(input_depth)

        xr = self.rgbd_fusion(xr,xd)

        xr = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')
        xd = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')

        i = 4 # las veces que se repite el resnet_layer y la fusion rgbd
        for j in range(i):
            n = 4 # las veces que se repite el resnet_layer
            for k in range(n):
                xr = self.resnet_layer(xr,64)
                xd = self.resnet_layer(xd,64)
            
            xr = self.rgbd_fusion(xr,xd)

       



In [6]:
model = My_model_class()